# LSTM Model


## Import Data
### Players from RPS_game.py

In [1]:
from Lib.RPS_game import play, mrugesh, abbey, quincy, kris, human, random_player
from Lib.RPS_encoding import RPSpair_encode, RPSpair_decode, RPSpair_encode_series
from Lib.Telepathic_player import telepath
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, LSTM, Embedding
import pandas as pd
from os import listdir
from os.path import isfile, join

## Read and Concatenate Games
Take all games and load into dataframe

In [2]:
import pandas as pd
from os import listdir
from os.path import isfile, join

path = "Games/"
all_game_paths = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
games_concat = pd.DataFrame(columns = ["Player","Opponent","Result","Winning_Play"])
games = []
for game_path in all_game_paths:
    #print(game_path)
    game_data = pd.read_csv(game_path, names = ["Player","Opponent","Result","Winning_Play"])
    games.append(game_data)
    games_concat = games_concat.append(game_data)
    
#print(len(game_data.index)) 
#game_data.head()  
print(len(games_concat.index))
games_concat.head()

50000


,Player,Opponent,Result,Winning_Play
0,P,R,1,P
1,S,P,1,S
2,R,P,0,S
3,S,S,0,R
4,R,P,0,S


# LSTM Training
### Parameters

In [4]:
seq_length = 100 #how far back the ltsm looks when training
#Model parameters
BATCH_SIZE = 64
VOCAB_SIZE = 9 # RPS pairs
EMBEDDING_DIM = 64
RNN_UNITS = 512

TRAINING_EPOCHS = 100

#Kris_Mrugesh settings
#seq_length = 100 
#BATCH_SIZE = 64
#VOCAB_SIZE = 3 
#EMBEDDING_DIM = 128
#RNN_UNITS = 512
#TRAINING_EPOCHS = 300


### Format data for LTSM training

In [5]:
#Predict the target combination that will occur next from prev combinations
#RR = 0
#RP = 1
#RS = 2
#PR = 3
#PP = 4
#PS = 5
#SR = 6
#SP = 7
#SS = 8




temp = games_concat.drop("Result", axis = 1)
temp.index = [x for x in range(temp.index.size)]
tempInput = pd.Series([temp.loc[x,"Player"] + temp.loc[x,"Opponent"] for x in range(temp['Player'].size)])
tempTarget = pd.Series([temp.loc[x,"Winning_Play"] + temp.loc[x,"Opponent"] for x in range(temp['Player'].size)])
print(tempInput)
print(tempTarget)
tempInput = RPSpair_encode_series(tempInput)
tempTarget = RPSpair_encode_series(tempTarget)
print(tempInput)
print(tempTarget)

0        PR
1        SP
2        RP
3        SS
4        RP
         ..
49995    SP
49996    PR
49997    SP
49998    RS
49999    PR
Length: 50000, dtype: object
0        PR
1        SP
2        SP
3        RS
4        SP
         ..
49995    SP
49996    PR
49997    SP
49998    RS
49999    PR
Length: 50000, dtype: object
0        3
1        7
2        1
3        8
4        1
        ..
49995    7
49996    3
49997    7
49998    2
49999    3
Length: 50000, dtype: int64
0        3
1        7
2        7
3        2
4        7
        ..
49995    7
49996    3
49997    7
49998    2
49999    3
Length: 50000, dtype: int64


In [6]:
input_data = tf.data.Dataset.from_tensor_slices(tempInput)
target_data = tf.data.Dataset.from_tensor_slices(tempTarget)

#split into sequences
input_sequences = input_data.batch(seq_length+1, drop_remainder = True)
target_sequences = target_data.batch(seq_length+1, drop_remainder = True)
print(input_sequences)

#remove last element from all input sequences
def remove_last(chunk):
    return chunk[:-1]
inputs = input_sequences.map(remove_last)

#remove first element from all target sequences
def remove_first(chunk):
    return chunk[1:]
targets = target_sequences.map(remove_first)

dataset = tf.data.Dataset.zip((inputs, targets))
for element in dataset:
    print(element)
    break


<BatchDataset shapes: (101,), types: tf.int64>
(<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([3, 7, 1, 8, 1, 0, 3, 0, 5, 4, 7, 7, 7, 7, 6, 0, 5, 3, 2, 2, 8, 0,
       5, 1, 8, 1, 2, 2, 2, 4, 7, 1, 8, 1, 8, 1, 8, 7, 7, 7, 1, 3, 4, 1,
       2, 2, 1, 4, 1, 7, 1, 7, 1, 6, 7, 7, 4, 7, 7, 7, 7, 6, 3, 4, 1, 3,
       4, 7, 6, 3, 4, 7, 6, 3, 1, 3, 4, 7, 6, 3, 1, 5, 1, 5, 1, 2, 2, 2,
       2, 4, 1, 8, 6, 0, 5, 4, 7, 6, 0, 5], dtype=int64)>, <tf.Tensor: shape=(100,), dtype=int64, numpy=
array([7, 7, 2, 7, 3, 3, 3, 2, 7, 7, 7, 7, 7, 3, 3, 2, 3, 2, 2, 2, 3, 2,
       7, 2, 7, 2, 2, 2, 7, 7, 7, 2, 7, 2, 7, 2, 7, 7, 7, 7, 3, 7, 7, 2,
       2, 7, 7, 7, 7, 7, 7, 7, 3, 7, 7, 7, 7, 7, 7, 7, 3, 3, 7, 7, 3, 7,
       7, 3, 3, 7, 7, 3, 3, 7, 3, 7, 7, 3, 3, 7, 2, 7, 2, 7, 2, 2, 2, 2,
       7, 7, 2, 3, 3, 2, 7, 7, 3, 3, 2, 7], dtype=int64)>)


### Build the model

In [7]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.Sequential()
    model.add(Embedding(vocab_size, embedding_dim, batch_input_shape = [batch_size, None]))
    model.add(LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size+1))
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 64)            576       
_________________________________________________________________
lstm (LSTM)                  (64, None, 512)           1181696   
_________________________________________________________________
dense (Dense)                (64, None, 10)            5130      
Total params: 1,187,402
Trainable params: 1,187,402
Non-trainable params: 0
_________________________________________________________________


### Create loss function

In [8]:
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits = True)

### Compile the model

In [9]:
model.compile(optimizer='adam', loss=loss)

### Create Checkpoints
configure the model to create checkpoints as it trains so the model can be reloaded and continue training

In [10]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath = checkpoint_prefix, save_weights_only = True)

print(tf.__version__)

2.3.1


In [11]:
#CALL TO LOAD LATEST CHECKPOINT FOR TRAINING
checkpoint_dir = './training_checkpoints'
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

### Training the model

In [12]:
#Shuffle sequences before batching
data = dataset.shuffle(10000).batch(BATCH_SIZE, drop_remainder = True)
history = model.fit(data, epochs = TRAINING_EPOCHS, callbacks = [checkpoint_callback])

Epoch 1/100
7/7 [==============================] - 0s 19ms/step - loss: 0.4411
Epoch 2/100
7/7 [==============================] - 0s 19ms/step - loss: 0.1763
Epoch 3/100
7/7 [==============================] - 0s 19ms/step - loss: 0.1509
Epoch 4/100
7/7 [==============================] - 0s 19ms/step - loss: 0.1374
Epoch 5/100
7/7 [==============================] - 0s 19ms/step - loss: 0.1263
Epoch 6/100
7/7 [==============================] - 0s 19ms/step - loss: 0.1234
Epoch 7/100
7/7 [==============================] - 0s 19ms/step - loss: 0.1145
Epoch 8/100
7/7 [==============================] - 0s 19ms/step - loss: 0.1135
Epoch 9/100
7/7 [==============================] - 0s 19ms/step - loss: 0.1090
Epoch 10/100
7/7 [==============================] - 0s 19ms/step - loss: 0.1068
Epoch 11/100
7/7 [==============================] - 0s 21ms/step - loss: 0.1045
Epoch 12/100
7/7 [==============================] - 0s 20ms/step - loss: 0.1041
Epoch 13/100
7/7 [==============================]

### Rebuild model for predicting one output rather than a vector

In [13]:
#change config
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size = 1)
#load trained weights from checkpoint
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

## Generate Outputs from Model

In [14]:
def Get_Prediction(model, opponent_history, player_history):
    input_raw = [x + y if x != '' and y != '' else 'PR' for (x,y) in zip(player_history, opponent_history)]
    #print(input_raw)
    #format history for input
    input_eval = [RPSpair_encode(c) for c in input_raw]
    input_eval = tf.expand_dims(input_eval, 0)
    
    #high temperature  => suprising output
    #low temperature => predictable output
    temperature = 1; 
    
    model.reset_states()
    
    prediction = model(input_eval)
    prediction = tf.squeeze(prediction, 0)
    
    prediction = prediction / temperature
    predicted_id = tf.random.categorical(prediction, num_samples = 1)[-1,0].numpy()
    #print(predicted_id)
    #print("Guessing: " + RPSpair_decode(predicted_id))
    return RPSpair_decode(predicted_id)[0]

#print(Get_Prediction(model, ['R','R','P']))

# RPS.py
Player function

In [15]:
plays = 1000

def player(prev_play, opponent_history = [], play_history=['']):
    if len(play_history) > plays: 
        opponent_history.clear()
        play_history.clear()
        play_history.append('')
    opponent_history.append(prev_play)
    #print("Actual: " + prev_play) 
    guess = Get_Prediction(model, opponent_history, play_history)
    #print(guess)
    play_history.append(guess)
    return guess

# MAIN.PY

In [17]:
#play(player, quincy, plays)
play(player, abbey, plays)
#play(player, kris, plays)
#play(player, mrugesh, plays)

Final results: {'p1': 319, 'p2': 464, 'tie': 217}
Player 1 win rate: 40.74074074074074%


40.74074074074074

## Saving the model

In [37]:
model.save('Models/LSTM_ImbalancedTrainingAbbey_GameHistoryBased_Abbey')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: Models/LSTM_ImbalancedTrainingAbbey_GameHistoryBased_Abbey\assets
